In [ ]:
import pandas as pd
import matplotlib as plt
import json
import os

In [ ]:
def split_path(path: str):
    # Remove .json
    path = path[:-5]
    parts = path.split("__")
    return {"strategy": parts[0], "dataset": parts[1], "pool_size": parts[2]}

def get_time_data(results):
    result_dfs = list()
    for path, datas in results.items():
        data = pd.concat([datas["solvers_times"], datas["trees_times"]])
        path_parts = split_path(path)
        data["strategy"] = path_parts["strategy"]
        data["dataset"] = path_parts["dataset"]
        data["pool_size"] = path_parts["pool_size"]

        result_dfs.append(data)
    return pd.concat(result_dfs) \
        .reset_index(names=["subject"], inplace=False) \
        .set_index(["dataset", "strategy", "pool_size", "subject"], inplace=False)


In [ ]:
target_paths = list()
for _, _, paths in os.walk("./evaluations/"):
    for path in paths:
        target_paths.append(path)


results = dict()
for path in target_paths:
    with open(f"./evaluations/{path}", "r") as f:
        results[path] = json.load(f)

results = {path: {key: pd.DataFrame(value) for key, value in data.items() if key != "cache"} for path, data in results.items()}
time_data = get_time_data(results)
time_data

In [ ]:
time_sum_data = time_data[["sum"]].pivot_table(index=["dataset", "strategy", "pool_size"], columns=["subject"])
time_sum_data.columns = time_sum_data.columns.droplevel(0)
time_sum_data = time_sum_data[["adding", "checking", "copying", "finding", "insertion"]]
time_sum_data = time_sum_data.pivot_table(index=["strategy", "pool_size"], columns=["dataset"])
time_sum_data.columns = time_sum_data.columns.swaplevel(0, 1)
for dataset in time_sum_data.columns.levels[0]:
    time_sum_data[dataset].plot.bar(stacked=True, title=dataset)